In [1]:
import os

import torch
from tqdm import tqdm
from ffcv.fields import BytesField, IntField, RGBImageField
from ffcv.writer import DatasetWriter

from data_utils.data_stats import *
from data_utils.dataloader import get_loader
from utils.metrics import topk_acc, real_acc, AverageMeter
from models.networks import get_model
from data_utils.dataset_to_beton import get_dataset

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms

import ast
import pickle

c:\Users\merci\miniconda3\envs\ffcv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
dataset = 'imagenet'                 # One of cifar10, cifar100, stl10, imagenet or imagenet21
architecture = 'B_12-Wi_1024'
data_resolution = 64                # Resolution of data as it is stored
crop_resolution = 64                # Resolution of fine-tuned model (64 for all models we provide)
num_classes = CLASS_DICT[dataset]
data_path = './beton/'
eval_batch_size = 1024
checkpoint = 'in21k_imagenet'  #'in21k_cifar100'        # This means you want the network pre-trained on ImageNet21k and finetuned on CIFAR10

DEVICE = "cuda:0"
mode = "pgd" #pgd or fgsm

In [20]:
torch.backends.cuda.matmul.allow_tf32 = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the model and specify the pre-trained weights
model = get_model(architecture=architecture, resolution=crop_resolution, num_classes=CLASS_DICT[dataset],
                  checkpoint=checkpoint)
model.cuda()

Weights already downloaded
Load_state output <All keys matched successfully>


BottleneckMLP(
  (linear_in): Linear(in_features=12288, out_features=1024, bias=True)
  (linear_out): Linear(in_features=1024, out_features=1000, bias=True)
  (blocks): ModuleList(
    (0-11): 12 x BottleneckBlock(
      (block): Sequential(
        (0): Linear(in_features=1024, out_features=4096, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=4096, out_features=1024, bias=True)
      )
    )
  )
  (layernorms): ModuleList(
    (0-11): 12 x LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
)

In [21]:
# Get the test loader
from torchvision.transforms import ToTensor
data_path = "C:\\mlp\\scaling_mlps\\beton"
loader = get_loader(
    dataset,
    bs=eval_batch_size,
    mode="test",
    augment=False,
    dev=device,
    mixup=0.0,
    data_path=data_path,
    data_resolution=data_resolution,
    crop_resolution=crop_resolution,
)

Loading C:\mlp\scaling_mlps\beton\imagenet\ffcv\val\val_64.beton


In [38]:
def pgd(model, x_batch, target, k, eps, eps_step):
    loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')

    # Initialize random input around x_batch within an eps sized box
    x_adv = x_batch + eps * (2*torch.rand_like(x_batch) - 1)
    # clamp values back down to the [0,1] range
    x_adv.clamp_(min=0., max=1.)
    
    for _ in range(k):
        x_adv.detach_().requires_grad_()

        # Calculate the loss and calculate the gradients.
        model.zero_grad()
        out = model(x_adv)
        loss_fn(out, target).backward()
    
        # Calculate the step and add the step and project back to the eps
        # sized box around x_batch
        step = eps_step * x_adv.grad.sign()
        x_adv = x_batch + (x_adv + step - x_batch).clamp_(min=-eps, max=eps)

        # clamp back to image domain; in contrast to the previous exercise we clamp at each step (so this is part of the projection)
        # both implementations are valid; this tents to work slightly better
        x_adv.clamp_(min=0, max=1)

    return x_adv.detach()


def fgsm_(model, x, target, eps, targeted=True, clip_min=None, clip_max=None):
    """Internal process for all FGSM and PGD attacks."""    
    # create a copy of the input, remove all previous associations to the compute graph...
    input_ = x.clone().detach_().to(DEVICE)
    # ... and make sure we are differentiating toward that variable
    input_.requires_grad_()

    # run the model and obtain the loss
    logits = model(input_).to(DEVICE)
    #target = torch.LongTensor([target]).to(DEVICE)
    model.zero_grad()
    loss = torch.nn.CrossEntropyLoss()(logits, target).to(DEVICE)
    loss.backward()
    
    #perform either targeted or untargeted attack
    if targeted:
        out = input_ - eps * input_.grad.sign()
    else:
        out = input_ + eps * input_.grad.sign()
    
    #if desired clip the ouput back to the image domain
    if (clip_min is not None) or (clip_max is not None):
        out.clamp_(min=clip_min, max=clip_max)
    return out

def fgsm_targeted(model, x, target, eps, **kwargs):
    return fgsm_(model, x, target, eps, targeted=True, **kwargs)

def fgsm_untargeted(model, x, label, eps, **kwargs):
    return fgsm_(model, x, label, eps, targeted=False, **kwargs)


In [35]:
# Define FGGSM
#@torch.no_grad()
def test_adversarial(model, loader, eps, mode, epsStep = None):
    model.eval()
    total_acc, total_top5 = AverageMeter(), AverageMeter()
    total_accFGGSM, total_top5FGGSM = AverageMeter(), AverageMeter()
    for ims, targs in tqdm(loader, desc="Evaluation"):

        ims = torch.reshape(ims, (ims.shape[0], -1)).cuda()
        targs = targs.to(DEVICE)

        if mode =="fgsm":
            imsFGSM = fgsm_(model, ims, targs, eps, targeted=True, clip_min=None, clip_max=None)

        if mode == "pgd":
            imsFGSM = pgd(model,ims,targs,1,eps,epsStep)

        preds = model(ims).cuda()
        
        predsFSM = model(imsFGSM).cuda()
   
        if dataset != 'imagenet_real':
            acc, top5 = topk_acc(preds, targs, k=5, avg=True)
            accFGGSM, top5FGGSM = topk_acc(predsFSM, targs, k=5, avg=True)

        else:
            acc = real_acc(preds, targs, k=5, avg=True)
            top5 = 0

        total_acc.update(acc, ims.shape[0])
        total_top5.update(top5, ims.shape[0])

        total_accFGGSM.update(accFGGSM, ims.shape[0])
        total_top5FGGSM.update(top5FGGSM, ims.shape[0])

    return (
        total_acc.get_avg(percentage=True),
        total_top5.get_avg(percentage=True),
        total_accFGGSM.get_avg(percentage=True),
        total_top5FGGSM.get_avg(percentage=True),
    )

In [39]:
test_acc, test_top5, total_accFGGSM, total_top5FGGSM = test_adversarial(model, loader,.005, mode = mode, epsStep = 0.1)

# Print all the stats
print("Test Accuracy        ", "{:.4f}".format(test_acc))
print("Top 5 Test Accuracy          ", "{:.4f}".format(test_top5))
print("Test Accuracy FG       ", "{:.4f}".format(total_accFGGSM))
print("Top 5 Test Accuracy  FG        ", "{:.4f}".format(total_top5FGGSM))

Evaluation: 100%|██████████| 49/49 [00:23<00:00,  2.12it/s]

Test Accuracy         43.0300
Top 5 Test Accuracy           66.6800
Test Accuracy FG        8.9960
Top 5 Test Accuracy  FG         20.5000
